# Phoneme seq2seq with CMUdict (encoder-decoder)
Train a Transformer encoder-decoder with cross-entropy to map noisy phoneme sequences back to clean phoneme sequences using LibriSpeech text.


In [1]:
from pathlib import Path
import math
import random
import re
import pickle
from typing import Dict, List, Optional, Tuple

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler


In [2]:
# Hyperparameters / config collected here for quick tuning
DATA_DIR = Path("data")
CMU_PATH = DATA_DIR / "cmudict-0.7b"
SENTENCE_PKL = DATA_DIR / "librispeech_sentences.pkl"

MAX_SENTENCES = 250_000
MAX_LEN = 128
TRAIN_SAMPLES = 200_000
VAL_SAMPLES = 10_000
BATCH_SIZE = 128
NUM_EPOCHS = 200
SEED = 0

NOISE = (0.1, 0.1, 0.1)  # (p_sub, p_ins, p_del)
D_MODEL = 512
N_HEAD = 8
NUM_ENCODER_LAYERS = 6
NUM_DECODER_LAYERS = 6
DIM_FEEDFORWARD = 2048
DROPOUT = 0.2

BASE_LR = 1e-3
END_LR = 1e-5
WEIGHT_DECAY = 1e-4
WARMUP_PCT = 0.1
GRAD_ACCUM_STEPS = 1
MIXED_PRECISION = True
USE_TF32 = True

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rng = random.Random(SEED)
torch.manual_seed(SEED)


In [3]:
# Load CMUdict and LibriSpeech sentences
if not CMU_PATH.exists():
    raise FileNotFoundError(f"Missing CMUdict at {CMU_PATH}")
if not SENTENCE_PKL.exists():
    raise FileNotFoundError(f"Missing LibriSpeech sentences at {SENTENCE_PKL}")

stress_digits = "0123456789"
cmu_map: Dict[str, List[str]] = {}
with CMU_PATH.open("r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith(";;;"):
            continue
        parts = line.split()
        word, phones = parts[0], parts[1:]
        clean = [ph.translate({ord(d): None for d in stress_digits}) for ph in phones]
        cmu_map[word.lower()] = clean

with SENTENCE_PKL.open("rb") as f:
    raw_sentences: List[str] = pickle.load(f)

len(cmu_map), len(raw_sentences)


(134373, 4500406)

In [4]:
# Build phoneme vocabulary (pad=0, sos=1, eos=2)
ALL_PHONEMES = sorted({ph for phones in cmu_map.values() for ph in phones})
PAD_IDX, SOS_IDX, EOS_IDX = 0, 1, 2
phoneme_to_idx: Dict[str, int] = {ph: i + 3 for i, ph in enumerate(ALL_PHONEMES)}
phoneme_to_idx["<pad>"] = PAD_IDX
phoneme_to_idx["<sos>"] = SOS_IDX
phoneme_to_idx["<eos>"] = EOS_IDX
idx_to_phoneme = {i: p for p, i in phoneme_to_idx.items()}

vocab_size = len(phoneme_to_idx)

CONFUSABLE_GROUPS = [
    ["DH", "TH"],
    ["IH", "IY"],
    ["AH", "AA", "AE"],
    ["EH", "AE"],
    ["S", "Z"],
    ["SH", "ZH"],
    ["P", "B"],
    ["T", "D"],
    ["K", "G"],
    ["F", "V"],
    ["CH", "JH"],
]
neighbor_map: Dict[int, List[int]] = {}
for group in CONFUSABLE_GROUPS:
    ids = [phoneme_to_idx[p] for p in group if p in phoneme_to_idx]
    for pid in ids:
        neighbor_map[pid] = [q for q in ids if q != pid]

vocab_size, list(phoneme_to_idx.items())[:5]


(42, [('AA', 3), ('AE', 4), ('AH', 5), ('AO', 6), ('AW', 7)])

In [5]:
WORD_RE = re.compile(r"[A-Za-z']+")

def sentence_to_phonemes(text: str) -> Optional[List[str]]:
    words = WORD_RE.findall(text.lower())
    if not words:
        return None
    phonemes: List[str] = []
    for w in words:
        phones = cmu_map.get(w)
        if not phones:
            return None
        phonemes.extend(phones)
    return phonemes

if MAX_SENTENCES and len(raw_sentences) > MAX_SENTENCES:
    sampled_sentences = rng.sample(raw_sentences, MAX_SENTENCES)
else:
    sampled_sentences = list(raw_sentences)

sentence_records: List[Dict[str, object]] = []
for sent in sampled_sentences:
    phonemes = sentence_to_phonemes(sent)
    if phonemes:
        sentence_records.append({"text": sent, "phonemes": phonemes})

if not sentence_records:
    raise RuntimeError("No sentences could be converted with CMUdict coverage.")
print(f"Prepared {len(sentence_records):,} sentence records (from {len(sampled_sentences):,} samples).")


Prepared 178,713 sentence records (from 250,000 samples).


In [6]:
def add_noise(
    seq: List[int],
    vocab_size: int,
    p_sub: float = 0.1,
    p_ins: float = 0.05,
    p_del: float = 0.05,
) -> List[int]:
    noisy: List[int] = []
    for token in seq:
        if random.random() < p_del:
            continue
        if random.random() < p_sub:
            neighbors = neighbor_map.get(token, [])
            if neighbors and random.random() < 0.8:
                token = random.choice(neighbors)
            else:
                token = random.randint(3, vocab_size - 1)
        noisy.append(token)
        if random.random() < p_ins:
            noisy.append(random.randint(3, vocab_size - 1))
    return noisy


def encode_phonemes(phonemes: List[str], max_len: int) -> List[int]:
    tokens = [phoneme_to_idx[p] for p in phonemes if p in phoneme_to_idx]
    tokens = tokens[: max_len - 2]
    tokens.append(EOS_IDX)
    return tokens


def pad_sequence(seq: List[int], max_len: int, pad_value: int = PAD_IDX) -> List[int]:
    return seq + [pad_value] * (max_len - len(seq))


class PhonemeCorrectionDataset(Dataset):
    def __init__(
        self,
        records: List[Dict[str, object]],
        max_len: int = MAX_LEN,
        num_samples: Optional[int] = None,
        noise: Tuple[float, float, float] = NOISE,
    ) -> None:
        self.records = records
        self.max_len = max_len
        self.num_samples = num_samples or len(records)
        self.noise = noise

    def __len__(self) -> int:
        return self.num_samples

    def __getitem__(self, idx: int):
        clean_ph: List[str] = self.records[idx % len(self.records)]["phonemes"]  # type: ignore[index]
        tgt_tokens = encode_phonemes(clean_ph, self.max_len)
        noisy_tokens = add_noise(
            tgt_tokens,
            vocab_size,
            p_sub=self.noise[0],
            p_ins=self.noise[1],
            p_del=self.noise[2],
        )
        noisy_tokens = noisy_tokens[: self.max_len]
        tgt_tokens = tgt_tokens[: self.max_len]
        return torch.tensor(noisy_tokens), torch.tensor(tgt_tokens)


def collate_batch(batch):
    srcs, tgts = zip(*batch)
    max_src = min(max(len(s) for s in srcs), MAX_LEN)
    max_tgt = min(max(len(t) for t in tgts) + 2, MAX_LEN)  # room for <sos>/<eos>

    padded_src = [pad_sequence(s.tolist()[:max_src], max_src) for s in srcs]
    tgt_in, tgt_out = [], []
    for tgt in tgts:
        tokens = tgt.tolist()[: max_tgt - 2]
        seq = [SOS_IDX] + tokens + [EOS_IDX]
        seq = seq[:max_tgt]
        seq += [PAD_IDX] * (max_tgt - len(seq))
        tgt_in.append(seq[:-1])
        tgt_out.append(seq[1:])

    return torch.tensor(padded_src), torch.tensor(tgt_in), torch.tensor(tgt_out)


# quick sanity check
_ds = PhonemeCorrectionDataset(sentence_records, max_len=32, num_samples=5)
for noisy, clean in _ds:
    print("noisy: ", [idx_to_phoneme[i] for i in noisy.tolist() if i != PAD_IDX])
    print("clean: ", [idx_to_phoneme[i] for i in clean.tolist() if i != PAD_IDX])
    break


noisy:  ['AH', 'L', 'AE', 'M', 'S', 'EH', 'D', 'AY', 'F', 'AO', 'R', 'DH', 'AH', 'TH', 'ER', 'D', 'T', 'AY', 'M', 'AH', 'T', 'AE', 'T', 'DH', 'NG', 'IH', 'S', 'OW', '<eos>', 'V']
clean:  ['AH', 'L', 'AE', 'M', 'S', 'EH', 'D', 'AY', 'F', 'AO', 'R', 'DH', 'AH', 'TH', 'ER', 'D', 'T', 'AY', 'M', 'B', 'AH', 'T', 'AE', 'T', 'DH', 'IH', 'S', 'M', 'OW', 'M', '<eos>']


In [7]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 5000) -> None:
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x + self.pe[:, : x.size(1)].to(x.device)


def generate_square_subsequent_mask(sz: int, device: torch.device) -> torch.Tensor:
    mask = torch.full((sz, sz), float('-inf'), device=device)
    mask = torch.triu(mask, diagonal=1)
    mask.fill_diagonal_(0.0)
    return mask


class Seq2SeqTransformer(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        d_model: int = D_MODEL,
        nhead: int = N_HEAD,
        num_encoder_layers: int = NUM_ENCODER_LAYERS,
        num_decoder_layers: int = NUM_DECODER_LAYERS,
        dim_feedforward: int = DIM_FEEDFORWARD,
        dropout: float = DROPOUT,
    ) -> None:
        super().__init__()
        self.src_emb = nn.Embedding(vocab_size, d_model, padding_idx=PAD_IDX)
        self.tgt_emb = nn.Embedding(vocab_size, d_model, padding_idx=PAD_IDX)
        self.pos_enc = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True,
        )
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True,
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, src: torch.Tensor, tgt_in: torch.Tensor) -> torch.Tensor:
        src_key_padding_mask = src == PAD_IDX
        tgt_key_padding_mask = tgt_in == PAD_IDX
        tgt_mask = generate_square_subsequent_mask(tgt_in.size(1), tgt_in.device)

        src_h = self.encoder(self.pos_enc(self.src_emb(src)), src_key_padding_mask=src_key_padding_mask)
        tgt_h = self.decoder(
            self.pos_enc(self.tgt_emb(tgt_in)),
            src_h,
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
            memory_key_padding_mask=src_key_padding_mask,
        )
        return self.proj(tgt_h)


In [8]:
# Training setup
train_samples = min(TRAIN_SAMPLES, len(sentence_records))
val_samples = min(VAL_SAMPLES, len(sentence_records))

train_ds = PhonemeCorrectionDataset(sentence_records, max_len=MAX_LEN, num_samples=train_samples, noise=NOISE)
val_ds = PhonemeCorrectionDataset(sentence_records, max_len=MAX_LEN, num_samples=val_samples, noise=NOISE)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

model = Seq2SeqTransformer(
    vocab_size=vocab_size,
    d_model=D_MODEL,
    nhead=N_HEAD,
    num_encoder_layers=NUM_ENCODER_LAYERS,
    num_decoder_layers=NUM_DECODER_LAYERS,
    dim_feedforward=DIM_FEEDFORWARD,
    dropout=DROPOUT,
)
model = model.to(DEVICE)

if DEVICE.type == "cuda" and USE_TF32:
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.AdamW(model.parameters(), lr=BASE_LR, weight_decay=WEIGHT_DECAY)
total_train_iters = max(1, NUM_EPOCHS * len(train_loader))
warmup_iters = max(1, int(WARMUP_PCT * total_train_iters))
min_lr_factor = END_LR / BASE_LR


def lr_lambda(step: int) -> float:
    if step < warmup_iters:
        return (step + 1) / warmup_iters
    remaining = max(total_train_iters - warmup_iters, 1)
    decay_step = step - warmup_iters
    decay_frac = max(0.0, 1.0 - decay_step / remaining)
    return max(min_lr_factor, decay_frac)


scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)
scaler = GradScaler(enabled=MIXED_PRECISION and DEVICE.type == "cuda")


C:\Users\johnn\AppData\Local\Temp\ipykernel_14368\917891412.py:43: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=MIXED_PRECISION and DEVICE.type == "cuda")


In [ ]:
def tokens_to_phonemes(tokens: List[int]) -> List[str]:
    return [idx_to_phoneme.get(int(t), f"<{t}>") for t in tokens if int(t) not in (PAD_IDX, SOS_IDX, EOS_IDX)]


def greedy_decode(src: torch.Tensor, max_len: int = MAX_LEN) -> List[int]:
    model.eval()
    src = src.unsqueeze(0).to(DEVICE)
    src_key_padding_mask = src == PAD_IDX
    memory = model.encoder(model.pos_enc(model.src_emb(src)), src_key_padding_mask=src_key_padding_mask)
    ys = torch.tensor([[SOS_IDX]], device=DEVICE)
    for _ in range(max_len):
        tgt_mask = generate_square_subsequent_mask(ys.size(1), ys.device)
        out = model.decoder(
            model.pos_enc(model.tgt_emb(ys)),
            memory,
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=None,
            memory_key_padding_mask=src_key_padding_mask,
        )
        logits = model.proj(out[:, -1, :])
        next_word = logits.argmax(dim=-1).unsqueeze(0)
        ys = torch.cat([ys, next_word], dim=1)
        if next_word.item() == EOS_IDX:
            break
    return ys.squeeze(0).tolist()


def log_val_example(example_idx: int = 0) -> None:
    model.eval()
    noisy, tgt = val_ds[example_idx % len(val_ds)]
    rec = val_ds.records[example_idx % len(val_ds)]
    text = rec.get('text', '') if isinstance(rec, dict) else ''
    pred_tokens = greedy_decode(noisy, max_len=MAX_LEN)

    noisy_ph = tokens_to_phonemes(noisy.tolist())
    tgt_ph = tokens_to_phonemes(tgt.tolist())
    pred_ph = tokens_to_phonemes(pred_tokens)

    print(f"[val example] noisy:        {' '.join(noisy_ph)}")
    print(f"[val example] target:       {' '.join(tgt_ph)}")
    print(f"[val example] corrected:    {' '.join(pred_ph) if pred_ph else '<empty>'}")
    if text:
        print(f"[val example] plain text:   {text}")


def run_epoch(loader, train: bool = True):
    model.train(train)
    total_loss = 0.0
    accum = GRAD_ACCUM_STEPS if train else 1
    if train:
        optimizer.zero_grad(set_to_none=True)
    num_batches = len(loader)

    for step, (src, tgt_in, tgt_out) in enumerate(loader):
        src, tgt_in, tgt_out = src.to(DEVICE), tgt_in.to(DEVICE), tgt_out.to(DEVICE)

        # autocast may not accept a device_type kwarg on all torch versions;
        # use enabled=scaler.is_enabled() which already reflects MIXED_PRECISION & CUDA
        with autocast(enabled=scaler.is_enabled()):
            logits = model(src, tgt_in)
            loss = criterion(logits.reshape(-1, vocab_size), tgt_out.reshape(-1))

        total_loss += loss.item()

        if train:
            loss = loss / accum
            scaler.scale(loss).backward()
            do_step = (step + 1) % accum == 0 or (step + 1) == num_batches
            if do_step:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)
                scheduler.step()
    return total_loss / len(loader)


for epoch in range(NUM_EPOCHS):
    train_loss = run_epoch(train_loader, train=True)
    val_loss = run_epoch(val_loader, train=False)
    print(f"epoch {epoch}: train_loss={train_loss:.4f} | val_loss={val_loss:.4f} | lr={scheduler.get_last_lr()[0]:.6f}")
    log_val_example(example_idx=epoch)


C:\Users\johnn\AppData\Local\Temp\ipykernel_14368\3875417296.py:59: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=scaler.is_enabled()):
c:\Users\johnn\Desktop\EC ENGR C143A\c143a-project\.venv\lib\site-packages\torch\nn\functional.py:6044: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
c:\Users\johnn\Desktop\EC ENGR C143A\c143a-project\.venv\lib\site-packages\torch\nn\functional.py:6044: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


epoch 0: train_loss=2.8867 | val_loss=2.5472 | lr=0.000050
[val example] noisy:        AH L N AE M S EH D AY F AO R ER D T AY K M B T AE TH IY TH S B M OW M N T K EY M AY T T AE P AH P Z AA N DH AH D AO R
[val example] target:       AH L AE M S EH D AY F AO R DH AH TH ER D T AY M B AH T AE T DH IH S M OW M AH N T K EY M AH L AY T T AE P AH P AA N DH AH D AO R
[val example] corrected:    AY M AE T M AY M AY D AH N D AH N T M AY M AY M AE T S T ER S T AH N D M AY M AY M AY M AY T ER M AY M AY M
[val example] plain text:   A lamb!" said I, for the third time, but, at this moment, came a light tap upon the door.
[val example] noisy:        AH L N AE M S EH D AY F AO R ER D T AY K M B T AE TH IY TH S B M OW M N T K EY M AY T T AE P AH P Z AA N DH AH D AO R
[val example] target:       AH L AE M S EH D AY F AO R DH AH TH ER D T AY M B AH T AE T DH IH S M OW M AH N T K EY M AH L AY T T AE P AH P AA N DH AH D AO R
[val example] corrected:    AY M AE T M AY M AY D AH N D AH N T M AY M AY M AE T 

In [ ]:
# Decode a few examples (greedy autoregressive)
model.eval()
with torch.no_grad():
    src_batch, tgt_batch, _ = next(iter(val_loader))
    src_batch, tgt_batch = src_batch.to(DEVICE), tgt_batch.to(DEVICE)

for i in range(3):
    noisy_tokens = [idx_to_phoneme[t.item()] for t in src_batch[i] if t.item() != PAD_IDX]
    clean_tokens = [idx_to_phoneme[t.item()] for t in tgt_batch[i] if t.item() not in (PAD_IDX, SOS_IDX, EOS_IDX)]
    pred_tokens = greedy_decode(src_batch[i].cpu(), max_len=MAX_LEN)
    pred = [idx_to_phoneme.get(int(t), f"<{t}>") for t in pred_tokens if int(t) not in (PAD_IDX, SOS_IDX, EOS_IDX)]
    print(f"Noisy    : {' '.join(noisy_tokens)}")
    print(f"Target   : {' '.join(clean_tokens)}")
    print(f"Pred     : {' '.join(pred)}")


In [ ]:
export_path = Path("/phoneme_seq2seq.pt")
export_path.parent.mkdir(parents=True, exist_ok=True)

checkpoint = {
    "state_dict": model.state_dict(),
    "phoneme_to_idx": phoneme_to_idx,
    "idx_to_phoneme": idx_to_phoneme,
    "pad_idx": PAD_IDX,
    "sos_idx": SOS_IDX,
    "eos_idx": EOS_IDX,
    "max_len": MAX_LEN,
    "model_kwargs": {
        "vocab_size": vocab_size,
        "d_model": D_MODEL,
        "nhead": N_HEAD,
        "num_encoder_layers": NUM_ENCODER_LAYERS,
        "num_decoder_layers": NUM_DECODER_LAYERS,
        "dim_feedforward": DIM_FEEDFORWARD,
        "dropout": DROPOUT,
    },
}
torch.save(checkpoint, export_path)
print(f"Saved phoneme seq2seq checkpoint to {export_path.resolve()}")
